In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip

--2022-12-21 17:10:25--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  6.90MB/s    in 0.8s    

2022-12-21 17:10:27 (6.90 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [ ]:
import torch
try:
    import torch_geometric
except ModuleNotFoundError:
    TORCH = torch.__version__.split("+")[0]
    CUDA = "cu" + torch.version.cuda.replace(".", "")
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

In [ ]:
import os
import random
import math
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torch import nn, optim
from torch_geometric.data import Dataset, Data
from torch_geometric.typing import Adj
from torch_geometric.nn import MessagePassing
from tqdm import tqdm
import collections

# 1) LightGCN

GNN is a general name for a set of models that considers the problem setup from a graph perspective and utilizes neural networks to make predictions. In GNN, entities are usually treated as nodes, and relationships between entities are described by edges. One could further supplement the graph with additional information by attaching node features and edge features to the graph. GNN generates embeddings while taking into account graph structures. 

Among all instances of GNN, LightGCN is one that delivers state-of-the-art empirical performance on benchmarks for recommendations

https://medium.com/stanford-cs224w/lightgcn-for-movie-recommendation-eb6d112f1e8

https://colab.research.google.com/drive/1VfP6JlWbX_AJnx88yN1tM3BYE6XAADiy?usp=sharing#scrollTo=G30yzPzf6A36

## 1.1 Data Loading

In [ ]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table(
    './ml-1m/users.dat', sep='::', header=None, names=unames, engine='python', encoding='latin-1')

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(
    './ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python', encoding='latin-1')

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table(
    './ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python', encoding='latin-1')

df_full = pd.merge(pd.merge(ratings, users), movies)
matrix_data = df_full.groupby(['user_id', 'movie_id'])['rating'].mean().unstack().fillna(0)
matrix_data.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
rating_threshold = 3
config_dict = {
    "num_samples_per_user": 100,
    "num_users": 200,

    "epochs": 100,
    "batch_size": 128,
    "lr": 0.001,
    "weight_decay": 0.1,

    "embedding_size": 64,
    "num_layers": 5,
    "K": 10,
    "mf_rank": 8,

    "minibatch_per_print": 100,
    "epochs_per_print": 1,

    "val_frac": 0.2,
    "test_frac": 0.1,

    "model_name": "model.pth"
}

epochs = config_dict["epochs"]
batch_size = config_dict["batch_size"]
lr = config_dict["lr"]
weight_decay = config_dict["weight_decay"]
K = config_dict["K"]

In [ ]:
def trans_ml(dat, thres):
    """
    Transform function that assign non-negative entries >= thres 1, and non-
    negative entries <= thres 0. Keep other entries the same.
    """
    thres = thres[0]
    matrix = dat['edge_index']
    matrix[(matrix < thres) & (matrix > -1)] = 0
    matrix[(matrix >= thres)] = 1
    dat['edge_index'] = matrix
    return dat

class MovieLens(Dataset):
    def __init__(self, data, config):
        super(MovieLens, self).__init__()
        self.data = data
        self.config = config

    def train_val_test_split(self, val_frac=0.2, test_frac=0.1):
        """
        Return two mask matrices (M, N) that represents edges present in the
        train and validation set
        """
    
        try:
            self.num_user, self.num_item
        except AttributeError:
            self.num_user = len(self.data["users"].unique())
            self.num_item = len(self.data["items"].unique())

        # get number of edges masked for training and validation
        num_train_replaced = round((test_frac + val_frac) * self.num_user * self.num_item)
        num_val_show = round(val_frac * self.num_user * self.num_item)

        # edges masked during training
        # list of int
        indices_user = np.random.randint(0, self.num_user, num_train_replaced)
        indices_item = np.random.randint(0, self.num_item, num_train_replaced)
        
        # sample part of edges from training stage to be unmasked during validation
        # list of int
        indices_val_user = np.random.choice(indices_user, num_val_show)
        indices_val_item = np.random.choice(indices_item, num_val_show)

        train_mask = torch.ones(self.num_user, self.num_item)
        train_mask[indices_user, indices_item] = 0

        val_mask = train_mask.clone()
        val_mask[indices_val_user, indices_val_item] = 1
        test_mask = torch.ones_like(train_mask)

        return train_mask, val_mask, test_mask

    def _sample_pos_neg(self, data, mask, num_samples_per_user):
        """
        Samples (user, positive item, negative item) tuples per user.
        If a user does not have a postive (negative) item, we choose an item
        with unknown liking (an item without raw rating data).

        Args:
            data: Dataset object containing edge_index and raw ratings matrix.
            mask: Masking matrix indicating edges present in the current set
            num_samples_per_user: Number of samples to generate for each user.

        Returns:
            torch.Tensor object of (user, positive item, negative item) samples.
        """

        samples = []
        all_items = set(range(len(data["items"])))

        for user_index, user in enumerate(data["users"]):
            
            # empty set
            pos_items = set(
                torch.nonzero(data["edge_index"][user_index])[:, 0].tolist())
            
            # all item
            unknown_items = all_items.difference(
                    set(torch.nonzero(data["raw_edge_index"][user_index])[:, 0].tolist()))

            # empty set   
            neg_items = all_items.difference(
                set(pos_items)).difference(set(unknown_items))

            # all item without mask
            unmasked_items = set(torch.nonzero(mask[user_index])[:, 0].tolist())

            if (len(unknown_items.union(pos_items)) == 0) or (len(unknown_items.union(neg_items)) == 0):
                continue
                                                                                                                                                   
            for _ in range(num_samples_per_user):

                if len(pos_items.intersection(unmasked_items)) == 0:
                    pos_item_index = random.choice(list(unknown_items.intersection(unmasked_items)))
                else:
                    pos_item_index = random.choice(list(pos_items.intersection(unmasked_items)))
                    
                if len(neg_items.intersection(unmasked_items)) == 0:
                    neg_item_index = random.choice(list(unknown_items.intersection(unmasked_items)))
                else:
                    neg_item_index = random.choice(list(neg_items.intersection(unmasked_items)))
                    
                samples.append((user_index, pos_item_index, neg_item_index))

        return torch.tensor(samples, dtype=torch.int32)

    def sample_pos_neg(self):
        """
        Args:
            data: Dataset object containing edge_index and raw ratings matrix.
            train_mask: Masking matrix indicating edges present in train set.
            val_mask: Masking matrix indicating edges present in validation set.
            test_mask: Masking matrix indicating edges present in test set.
            num_samples_per_user: Number of samples to generate for each user.

        Returns:
            torch.Tensor object of (user, positive item, negative item) samples for
            train, validation and test.
        """

        config = self.config
        train_mask, val_mask, test_mask = self.train_val_test_split(
            config['val_frac'], config['test_frac']
        )

        train_samples = self._sample_pos_neg(self.data, train_mask, config['num_samples_per_user'])
        val_samples = self._sample_pos_neg(self.data, val_mask, config['num_samples_per_user'])
        test_samples = self._sample_pos_neg(self.data, test_mask, config['num_samples_per_user'])
        return train_samples, val_samples, test_samples, train_mask, val_mask, test_mask

# create Data object
data = Data(
    edge_index=torch.Tensor(matrix_data.to_numpy()),
    raw_edge_index=torch.Tensor(matrix_data.to_numpy()).clone(),
    data=ratings,
    users=users['user_id'],
    items=movies['movie_id'],
    )

# pre-process
data = trans_ml(data, [rating_threshold])
movie_dataset = MovieLens(data, config_dict)
(samples_train, samples_val, samples_test, train_mask, val_mask, test_mask) = \
    movie_dataset.sample_pos_neg()

print(samples_train.size())

torch.Size([604000, 3])


## 1.2 Implementation

### 1.2.1 LightGCN neighbourhood aggregation layer

Starting with the initial embeddings $E^{(0)}$ and the bipartite graph, we iterate over each node to perform neighborhood aggregation. Note that LightGCN uses **a simple weighted sum aggregator** and **avoids the heavy-lifting feature transformation and nonlinear activation**.

Within each layer, for each user in the graph, we compute its updated embedding as the weighted sum of embeddings from all its neighboring items (movies) following the formula below:
$$ \textbf{e}_u^{(k+1)} = \sum_{i \in N_u} \frac{1}{\sqrt{|N_u|} \sqrt{|N_i|}} \textbf{e}_i^{(k)} $$
where $ \textbf{e}_u^{(k)} $ and $ \textbf{e}_i^{(k)} $ are the user and item (movie) node embeddings at the k-th layer. $ |N_u| $ and $ |N_i| $ are the user and item nodes’ number of neighbors.

Similarly, for each item, the updated embedding is computed using weighted sum of its neighboring users:
$$ \textbf{e}_i^{(k+1)} = \sum_{i \in N_i} \frac{1}{\sqrt{|N_i|} \sqrt{|N_u|}} \textbf{e}_u^{(k)} $$

In [ ]:
class LightGCNConv(MessagePassing):
    """
    Args:
        in_channels (int): Size of each input sample.
        out_channels (int): Size of each output sample.
        num_users (int): Number of users for recommendation.
        num_items (int): Number of items to recommend.
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.MessagePassing`.
    """
    def __init__(
        self, in_channels: int, out_channels: int,num_users: int, num_items: int, **kwargs
        ):

        super(LightGCNConv, self).__init__(**kwargs)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_users = num_users
        self.num_items = num_items

    def forward(self, x, edge_index):
        """Performs neighborhood aggregation for user/item embeddings."""

        user_item = torch.zeros(self.num_users, self.num_items, device=x.device)
        user_item[edge_index[:, 0], edge_index[:, 1]] = 1
        user_neighbor_counts = torch.sum(user_item, axis=1)
        item_neightbor_counts = torch.sum(user_item, axis=0)

        # Compute weight for aggregation: 1 / sqrt(N_u * N_i)
        weights = user_item / torch.sqrt(
                user_neighbor_counts.repeat(self.num_items, 1).T \
                * item_neightbor_counts.repeat(self.num_users, 1))
        
        weights = torch.nan_to_num(weights, nan=0)
        out = torch.concat(
            (
                weights.T @ x[:self.num_users],
             weights @ x[self.num_users:]), 
             0)
        return out

### 1.2.2 LightGCN model

At layer combination, instead of taking the embedding of the final layer, LightGCN computes **a weighted sum of the embeddings at different layers**:
$$ \textbf{e}_u = \sum_{k=0}^K \alpha_k \textbf{e}_u^{(k)} $$
$$ \textbf{e}_i = \sum_{k=0}^K \alpha_k \textbf{e}_i^{(k)} $$
with $ \alpha \ge 0 $. Here, alpha values can either be learned as network parameters, or set as empirical hyperparameters. It has been found that $ \alpha = \frac{1}{K + 1} $ works well.

LightGCN predicts based on the inner product of the final user and item (movie) embeddings:
$$ \hat{y}_{ui} = \textbf{e}_u^T \textbf{e}_i $$
This inner product measures the similarity between the user and movie, therefore allowing us to understand how likely it is for the user to like the movie.

In [ ]:
class LightGCN(nn.Module):
    def __init__(
        self, 
        config: dict,
        device=None,
        **kwargs):

        super().__init__()

        self.num_users  = config["n_users"]
        self.num_items  = config["m_items"]
        self.embedding_size = config["embedding_size"]
        self.in_channels = self.embedding_size
        self.out_channels = self.embedding_size
        self.num_layers = config["num_layers"]

        # 0-th layer embedding.
        self.embedding_user_item = torch.nn.Embedding(
            num_embeddings=self.num_users + self.num_items,
            embedding_dim=self.embedding_size)
        self.alpha = None

        # random normal init seems to be a better choice when lightGCN actually
        # don't use any non-linear activation function
        nn.init.normal_(self.embedding_user_item.weight, std=0.1)
        print('use NORMAL distribution initilizer')

        self.f = nn.Sigmoid()

        self.convs = nn.ModuleList()
        self.convs.append(LightGCNConv(
                self.embedding_size, self.embedding_size,
                num_users=self.num_users, num_items=self.num_items, **kwargs))

        for _ in range(1, self.num_layers):
            self.convs.append(
                LightGCNConv(
                        self.embedding_size, self.embedding_size, 
                        num_users=self.num_users, num_items=self.num_items,
                        **kwargs))

        self.device = None
        if device is not None:
            self.convs.to(device)
            self.device = device

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, edge_index, *args, **kwargs):
        xs = []

        edge_index = torch.nonzero(edge_index)
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index, *args, **kwargs)
            if self.device is not None:
                x = x.to(self.device)
            xs.append(x)
        xs = torch.stack(xs)
        
        self.alpha = 1 / (1 + self.num_layers) * torch.ones(xs.shape)
        if self.device is not None:
            self.alpha = self.alpha.to(self.device)
            xs = xs.to(self.device)
        x = (xs * self.alpha).sum(dim=0)  # Sum along K layers.
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_config = {
    "n_users": len(users['user_id']),
    "m_items": len(movies['movie_id']),
    "embedding_size": config_dict["embedding_size"],
    "num_layers": config_dict["num_layers"],
}

### 1.2.3 Bayesian Personalized Ranking loss (BPR loss)

BPR attempts to learn the correct rank-ordering of items for each user by maximizing the posterior probability (MAP) of the model parameters given a data set of observed user-item preferences and a chosen prior distribution. Each user’s observed items (implicit feedback) are assumed to be preferred over the unobserved items.

To train the LightGCN model, we need an objective function that aligns with our goal for movie recommendation. We use the Bayesian Personalized Ranking (BPR) loss, which encourages observed user-item predictions to have increasingly higher values than unobserved ones, along with $ L_2 $ regularization:
$$ L_{BPR} = - \sum_{u=1}^M \sum_{i \in N_u} \sum_{j \notin N_u} \ln \sigma(\hat{y}_{ui} - \hat{y}_{uj}) + \lambda ||\textbf{E}^{(0)} ||^2 $$
where $ \textbf{E}^{(0)} $ is a matrix with column vectors being the 0-th layer embeddings to learn.

In [ ]:
def getEmbedding(model, users, pos, neg, data, mask):
    """
    INPUT:
        model: the LightGCN model you are training on
        users: this is the user index (note: use 0-indexed and not user number)
        pos: positive index corresponding to an item that the user like
        neg: negative index corresponding to an item that the user doesn't like
        data: the entire data, used to fetch all users and all items
        mask: Masking matrix indicating edges present in the current set
    """

    # assuming we always search for users and items by their indices
    all_users_items = model(
        model.embedding_user_item.weight.clone(), data["edge_index"] * mask)
    all_users = all_users_items[:len(data["users"])]
    all_items = all_users_items[len(data["users"]):]
    users_emb = all_users[users]
    pos_emb = all_items[pos]
    neg_emb = all_items[neg]
    n_user = len(data["users"])
    users_emb_ego = model.embedding_user_item(users)
    # offset the index to fetch embedding from user_item
    pos_emb_ego = model.embedding_user_item(pos + n_user)
    neg_emb_ego = model.embedding_user_item(neg + n_user)
    return users_emb, pos_emb, neg_emb, users_emb_ego, pos_emb_ego, neg_emb_ego

def getUsersRating(model, users, data):
    """ Get the embedding of users
    INPUT:
        model: the LightGCN model you are training on
        users: this is the user index (note: use 0-indexed and not user number
        data: the entire data, used to fetch all users and all items
    """
    all_users_items = model(
        model.embedding_user_item.weight.clone(), data["edge_index"])
    all_users = all_users_items[:len(data["users"])]
    items_emb = all_users_items[len(data["users"]): ]
    users_emb = all_users[users.long()]
    rating = model.f(torch.matmul(users_emb, items_emb.t()))
    return rating

def bpr_loss(model, users, pos, neg, data, mask):
    """ 
    INPUT:
        model: the LightGCN model you are training on
        users: this is the user index (note: use 0-indexed and not user number)
        pos: positive index corresponding to an item that the user like
            (0-indexed, note to index items starting from 0)
        neg: negative index corresponding to an item that the user doesn't like
        data: the entire data, used to fetch all users and all items
        mask: Masking matrix indicating edges present in the current set
    OUTPUT:
        loss, reg_loss
    """

    assert len(users) == len(pos) and len(users) == len(neg)
    (users_emb, pos_emb, neg_emb, userEmb0,  posEmb0, negEmb0) = getEmbedding(
        model, users.long(), pos.long(), neg.long(), data, mask)
    
    reg_loss = (1/2) * (
        userEmb0.norm(2).pow(2) + 
        posEmb0.norm(2).pow(2)  + 
        negEmb0.norm(2).pow(2))/float(len(users)
        )

    pos_scores = torch.mul(users_emb, pos_emb)
    pos_scores = torch.sum(pos_scores, dim=1)
    neg_scores = torch.mul(users_emb, neg_emb)
    neg_scores = torch.sum(neg_scores, dim=1)
    loss = torch.mean(torch.nn.functional.softplus(neg_scores - pos_scores))
    return loss, reg_loss

### testing
lightGCN = LightGCN(model_config, device=device)
optimizer = optim.Adam(lightGCN.parameters(), lr=lr)
users = samples_train[:, 0:1]
pos = samples_train[:, 1:2]
neg = samples_train[:, 2:3]

loss, reg_loss = bpr_loss(
    lightGCN, users, pos, neg, data, train_mask)
reg_loss = reg_loss * weight_decay
loss = loss + reg_loss
# loss_sum += loss.detach()
# loss.backward()
# optimizer.step()

use NORMAL distribution initilizer


RuntimeError: ignored

### 1.2.4 Training

In [ ]:
### training epoch
epochs_tracked = []
train_topks = []
val_topks = []

for epoch in range(epochs):
    print("Training on the {} epoch".format(epoch))
    lightGCN.train()
    loss_sum = 0
    # Shuffle the order of rows.
    samples_train = samples_train[torch.randperm(samples_train.size()[0])]
    for batch_idx in range(math.ceil(len(samples_train) / batch_size)):
        optimizer.zero_grad()

        current_batch = samples_train[batch_idx*batch_size: (batch_idx+1)*batch_size]

        # Shuffle the order of rows.
        current_batch = current_batch[torch.randperm(current_batch.size()[0])]
        users = current_batch[:, 0:1]
        pos = current_batch[:, 1:2]
        neg = current_batch[:, 2:3]

        loss, reg_loss = bpr_loss(
            lightGCN, users, pos, neg, data, train_mask)
        reg_loss = reg_loss * weight_decay
        loss = loss + reg_loss
        loss_sum += loss.detach()

        loss.backward()
        optimizer.step()

        if batch_idx % config_dict["minibatch_per_print"] == 0:
            all_users = torch.linspace(start=0, end=n_users - 1, steps=n_users).long()
            user_indices = current_batch[:, 0]
            user_indices = user_indices.repeat(2).long()
            item_indices = torch.cat((current_batch[:, 1], current_batch[:, 2])).long()

            pred = getUsersRating(
                lightGCN, all_users, data)[user_indices, item_indices]

            truth = data["edge_index"][user_indices, item_indices]
            topk_precision, topk_recall = \
                personalized_topk(pred, K, user_indices, data["edge_index"])

    if epoch % config_dict["epochs_per_print"] == 0:
        epochs_tracked.append(epoch)

        # evaluation on both the trainisng and validation set
        lightGCN.eval()
        # predict on the training set

        users = samples_train[:, 0:1]
        user_indices = samples_train[:, 0]
        user_indices = user_indices.repeat(2).long()
        item_indices = torch.cat((samples_train[:, 1], samples_train[:, 2])).long()

        pred = getUsersRating(
            lightGCN, users[:,0], data)[user_indices, item_indices]

        truth = data["edge_index"][users.long()[:,0]][user_indices, item_indices]

        train_topk_precision, train_topk_recall = personalized_topk(
            pred, K, user_indices, data["edge_index"])
        
        train_topks.append((train_topk_precision, train_topk_recall))

        # predict on the validation set
        users_val = samples_val[:, 0:1]
        pos_val = samples_val[:, 1:2]
        neg_val = samples_val[:, 2:3]
        loss_val, reg_loss_val = bpr_loss(
            lightGCN, users_val, pos_val, neg_val, data, val_mask)
        reg_loss_val = reg_loss_val * weight_decay

        # predict on the validation set
        user_indices = samples_val[:, 0]
        user_indices = user_indices.repeat(2).long()
        item_indices = torch.cat((samples_val[:, 1], samples_val[:, 2])).long()
        pred_val = getUsersRating(
            lightGCN, users_val[:,0], data)[user_indices, item_indices]

        truth_val = data["edge_index"][users_val.long()[:,0]][user_indices, item_indices]
        val_topk_precision, val_topk_recall = personalized_topk(
            pred_val, K, user_indices, data["edge_index"])
        val_topks.append((val_topk_precision, val_topk_recall))

### 1.2.5 Evaluation


In [ ]:
def personalized_topk(pred, K, user_indices, edge_index):
    """Computes TopK precision and recall.

    Args:
        pred: Predicted similarities between user and item.
        K: Number of items to rank.
        user_indices: Indices of users for each prediction in `pred`.
        edge_index: User and item connection matrix.

    Returns:
        Average Top K precision and recall for users in `user_indices`.
    """
    per_user_preds = collections.defaultdict(list)
    for index, user in enumerate(user_indices):
        per_user_preds[user.item()].append(pred[index].item())
    precisions = 0.0
    recalls = 0.0
    for user, preds in per_user_preds.items():
        while len(preds) < K:
            preds.append(random.choice(range(edge_index.shape[1])))
        top_ratings, top_items = torch.topk(torch.tensor(preds), K)
        correct_preds = edge_index[user, top_items].sum().item()
        total_pos = edge_index[user].sum().item()
        precisions += correct_preds / K
        recalls += correct_preds / total_pos if total_pos != 0 else 0
    num_users = len(user_indices.unique())
    return precisions / num_users, recalls / num_users

In [ ]:
# predict on the test set
lightGCN.eval()
print("Training completed after {} epochs".format(epochs))

users_test = samples_test[:, 0:1]
pos_test = samples_test[:, 1:2]
neg_test = samples_test[:, 2:3]

loss_test, reg_loss_test = bpr_loss(
    lightGCN, users_test, pos_test, neg_test, data, test_mask)
reg_loss_test = reg_loss_test * weight_decay

# predict on the test set
user_indices = samples_test[:, 0]
user_indices = user_indices.repeat(2).long()
item_indices = torch.cat((samples_test[:, 1], samples_test[:, 2])).long()
pred_test = getUsersRating(lightGCN, users_test[:,0], data)\
    [user_indices, item_indices]
truth_test = data["edge_index"][users_test.long()[:,0]]\
    [user_indices, item_indices]
test_topk_precision, test_topk_recall = personalized_topk(
    pred_test, K, user_indices, data["edge_index"])

In [ ]:
# Compute baseline metrics using matrix factorization.
baseline_pred = matrix_factorization(
        data["edge_index"].detach().cpu().numpy(),
        config_dict["mf_rank"])[user_indices.cpu(), item_indices.cpu()]
baseline_topk_precision, baseline_topk_recall = \
        personalized_topk(baseline_pred, K, user_indices, data["edge_index"])
print("Baseline (PARAFAC matrix factorization) produces ",
      "Top K precision = {}, recall = {}.".format(baseline_topk_precision,
                                                  baseline_topk_recall))